Quantum ML model running on the full dataset  using default qubit  and full gradient method


In [ ]:
!pip install pennylane pennylane-qiskit scikit-learn pandas numpy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import pennylane as qml
from pennylane import numpy as np_pennylane  # Use PennyLane's NumPy for autograd
import time

start_time = time.time()

df = pd.read_csv('BankChurners.csv')  # Replace with your file path if needed

le = LabelEncoder()
df['Attrition_Flag_encoded'] = le.fit_transform(df['Attrition_Flag'])  # 0: Attrited, 1: Existing
df['Gender_encoded'] = le.fit_transform(df['Gender'])
df['Education_Level_encoded'] = le.fit_transform(df['Education_Level'])
df['Marital_Status_encoded'] = le.fit_transform(df['Marital_Status'])
df['Income_Category_encoded'] = le.fit_transform(df['Income_Category'])
df['Card_Category_encoded'] = le.fit_transform(df['Card_Category'])

features = ['Total_Trans_Ct', 'Total_Trans_Amt', 'Total_Relationship_Count', 'Total_Revolving_Bal']
X = df[features].values
y = df['Attrition_Flag_encoded'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")

n_qubits = len(features)  # 4 qubits for 4 features
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def quantum_circuit(inputs, weights):
    # Encode inputs using RY rotations (scale inputs to [0, π])
    for i in range(n_qubits):
        qml.RY(inputs[i] * np.pi, wires=i)  # Scale inputs for valid rotation angles

    # Variational layers
    for layer in range(3):
        for i in range(n_qubits):
            qml.RZ(weights[layer, i, 0], wires=i)
            qml.RY(weights[layer, i, 1], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])

    return qml.expval(qml.PauliZ(0))

n_layers = 3
weight_shapes = {"weights": (n_layers, n_qubits, 2)}
weights_init = np_pennylane.random.uniform(0, 2 * np.pi, weight_shapes["weights"], requires_grad=True)

# Define cross-entropy cost function
def cost(weights, X, y):
    predictions = np_pennylane.array([quantum_circuit(x, weights) for x in X], requires_grad=True)
    predictions = (predictions + 1) / 2  # Map to [0, 1]
    eps = 1e-15  # Avoid log(0)
    loss = -np_pennylane.mean(y * np_pennylane.log(predictions + eps) + (1 - y) * np_pennylane.log(1 - predictions + eps))
    return loss

opt = qml.AdamOptimizer(stepsize=0.05)
weights = weights_init
steps = 50  # Reduced steps to keep execution under 4 minutes

for it in range(steps):
    weights = opt.step(lambda w: cost(w, X_train, y_train), weights)
    if it % 10 == 0:  # Print less frequently to save time
        current_cost = cost(weights, X_train, y_train)
        print(f"Step {it}, Cost: {current_cost:.4f}")

# Predict on test set
predictions = np_pennylane.array([quantum_circuit(x, weights) for x in X_test])
predictions = (predictions + 1) / 2  # Map to [0, 1]
y_pred_class = (predictions > 0.5).astype(int)

# Evaluate
accuracy = accuracy_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"F1 Score: {f1:.4f}")

# End timing
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.2f} seconds")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 k